In [5]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import VGG16

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

import keras
from shutil import copyfile
from zipfile import ZipFile

import matplotlib.pyplot as plt
import scipy.io
from keras.preprocessing import image as kimage

In [6]:
NAMES = [
    "pink primrose", "hard-leaved pocket orchid", "canterbury bells",
    "sweet pea", "english marigold", "tiger lily", "moon orchid",
    "bird of paradise", "monkshood", "globe thistle", "snapdragon",
    "colt's foot", "king protea", "spear thistle", "yellow iris",
    "globe-flower", "purple coneflower", "peruvian lily", "balloon flower",
    "giant white arum lily", "fire lily", "pincushion flower", "fritillary",
    "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers",
    "stemless gentian", "artichoke", "sweet william", "carnation",
    "garden phlox", "love in the mist", "mexican aster", "alpine sea holly",
    "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip",
    "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia",
    "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy",
    "common dandelion", "petunia", "wild pansy", "primula", "sunflower",
    "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia",
    "pink-yellow dahlia?", "cautleya spicata", "japanese anemone",
    "black-eyed susan", "silverbush", "californian poppy", "osteospermum",
    "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania",
    "azalea", "water lily", "rose", "thorn apple", "morning glory",
    "passion flower", "lotus", "toad lily", "anthurium", "frangipani",
    "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow",
    "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum",
    "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow",
    "mexican petunia", "bromelia", "blanket flower", "trumpet creeper",
    "blackberry lily"
]

In [3]:
labels = scipy.io.loadmat('imagelabels.mat')
labels = labels['labels'][0]

split_index = scipy.io.loadmat('setid.mat')
test_id = split_index['trnid'][0]
val_id = split_index['valid'][0]
train_id = split_index['tstid'][0]
test_val_id = np.concatenate([test_id,val_id])

FileNotFoundError: [Errno 2] No such file or directory: 'imagelabels.mat'

In [ ]:
source = "102flowers_test"
dest1 = "102flowers_train"
dest2 = "102flowers_val"

list_files = os.listdir(source)
for i in range(len(list_files)):
    print ('image', i)
    if i not in (train_id-1):
        if i in (test_id-1):
            if not os.path.exists(dest1):
                os.makedirs(dest1)
            shutil.move(source+ '/'+ list_files[i], dest1 + '/' + list_files[i])
        else:
            if not os.path.exists(dest2):
                os.makedirs(dest2)
            shutil.move(source+ '/'+ list_files[i], dest2 + '/' + list_files[i])

In [170]:
def from_image_to_array (source):
    list_image = []
    for file_image in os.listdir(source):
        path_image = source + '/' + file_image
        img = kimage.load_img(path_image, target_size = (224,224))
        image_array = kimage.img_to_array(img)
        list_image.append(image_array)
    data = np.array(list_image)
    return data

In [171]:
X_train = from_image_to_array('102flowers_train')
X_train.shape

In [ ]:
X_test= from_image_to_array('102flowers_test')
X_test.shape

In [ ]:
X_val = from_image_to_array('102flowers_val')
X_val.shape

In [ ]:
y_train = labels[sorted(train_id-1)] # -1 perchè gli indici partono da 1 e non da 0
y_val = labels[sorted(val_id-1)]
y_test = labels[sorted(test_id-1)]
y_test_val = labels[sorted(test_val_id-1)]

In [ ]:
with open('102flowers_y_train.pickle', 'wb') as handle:
    pickle.dump(y_train, handle)
with open('102flowers_y_test.pickle', 'wb') as handle:
    pickle.dump(y_test, handle)
with open('102flowers_y_val.pickle', 'wb') as handle:
    pickle.dump(y_val, handle)
with open('102flowers_y_test_val.pickle', 'wb') as handle:
    pickle.dump(y_test_val, handle)

In [ ]:
import pickle
with open('102flowers_X_val.pickle', 'wb') as handle:
    pickle.dump(X_val, handle)
with open('102flowers_X_train.pickle', 'wb') as handle:
    pickle.dump(X_train, handle)
with open('102flowers_X_test.pickle', 'wb') as handle:
    pickle.dump(X_test, handle)

In [ ]:
source = "mask_NEW_train"
dest1 = "mask_NEW_test"
dest2 = "mask_NEW_val"

list_files = os.listdir(source)
for i in range(len(list_files)):
    print ('image', i)
    if i not in (train_id-1):
        if i in (test_id-1):
            if not os.path.exists(dest1):
                os.makedirs(dest1)
            shutil.move(source+ '/'+ list_files[i], dest1 + '/' + list_files[i])
        else:
            if not os.path.exists(dest2):
                os.makedirs(dest2)
            shutil.move(source+ '/'+ list_files[i], dest2 + '/' + list_files[i])

In [ ]:
source = 'mask_NEW_folders/'

list_files = os.listdir(source)
for i in range(len(list_files)):
    print(i)
    label = labels[i]
    label_name = NAMES[label-1]
    print(label_name)
    if not os.path.exists(source + label_name):
        print('sto creando la cartella', source + label_name)
        os.makedirs(source + label_name)
    shutil.move(source+ list_files[i], source + label_name + '/' + list_files[i])
    print('sto muovendo il file',list_files[i], 'dalla cartella', source, 'alla cartella', source + label_name)


In [ ]:
listdir = os.listdir('mask_NEW_folders')
for directory in listdir:
    listfiles = os.listdir('mask_NEW_folders/'+directory)
    for image in listfiles:
        image_path = 'mask_NEW_folders/'+directory+'/'+image
        img = kimage.load_img(image_path, target_size = (224,224))
        img.save ('mask_NEW_folders/'+directory+'/'+image)

In [ ]:
listfiles = os.listdir('mask_NEW')

for image in listfiles:
    image_path = 'mask_NEW/'+image
    img = kimage.load_img(image_path, target_size = (224,224))
    img.save ('mask_NEW/'+image)